In [68]:
import numpy as np
import pandas as pd
import sqlite3
from textblob import TextBlob
import re
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')


import spacy
nlp = spacy.load('en')
#Reviews containing the following words are reissued albums

def parse_content_for_reissue(df, list_of_words):
    df['reissue'] = 0
    for each in list_of_words:
        df['reissue'] = df['reissue'] + df['abstract'].str.contains(each).astype('int')
    df['reissue'] = df['reissue'] + df['best_new_reissue'].astype('int')
    df['reissue'] = df['reissue'].astype(bool).astype(int)

#Reviews that were reviewed years after their release are classic albums
def reissue_columns(df):
    df['num_years_since_release'] = df.pub_year - df.year
    df['new_album'] = ((df['reissue'] == 0) & (df['num_years_since_release'] <= 1)).astype(int)
    return df


#Remove unusual characters from content and abstract

def remove_weird_char(string):
    return re.sub('[^A-Za-z0-9]+', ' ', string)


#Add columns of adjectives and adverbs used in review content and abstract

def parse_for_adj(df, column, new_column_name):
    prop_noun_parse = []
    for i in range(len(df[column])):
        non_PN = []
        blob = nlp(unicode(df[column][i]))
        for each in blob:
            if each.tag_ == u'ADJ' or each.tag_ == u'JJ':
                non_PN.append(str(each))
        prop_noun_parse.append((df['reviewid'][i], " ".join(non_PN)))

    df_new = pd.DataFrame(prop_noun_parse, columns =['reviewid', new_column_name])
    return df.merge(df_new)


#Adding Sentiment Scores with TextBlob of content and abstract adjectives/adverbs. This will come in handy later.
def add_sentiment(df, column, new_col_pol, new_col_subj):
    sentiments = []
    for each in range(len(df['reviewid'])):
        sentiments.append((df['reviewid'].iloc[each], TextBlob(df[column][each]).sentiment[0], TextBlob(df[column][each]).sentiment[1]))
    #                           TextBlob(df_mid['content'][each], analyzer=NaiveBayesAnalyzer()).sentiment))

    df_cont_sent = pd.DataFrame(sentiments, columns=['reviewid', new_col_pol, new_col_subj])
    return df.merge(df_cont_sent)



#Add a few additional columns for more NLP and bins for scores
def add_columns(df):
    df['word_count'] = df['content'].str.count('\w+')
    df['adj_count'] = df['content_adj'].str.count('\w+')
    df['adj_freq'] = df['adj_count'] / df['word_count']
    df['score_bin'] = (df['score'] // 1).astype(int)
    return df

In [92]:
if __name__ == "__main__":
    #connect to databse
    nlp = spacy.load('en')
    conn = sqlite3.connect("../pitchfork-data/pitchfork_2017.db")
    #create dataframes for each table
    df = pd.read_sql_query("select * from reviews;", conn)
    df_reviews = pd.read_sql_query("select * from content", conn)
    df_years = pd.read_sql_query("select * from years;", conn)
    df_genres = pd.read_sql_query("select * from genres", conn)
    #create dummies to allow for multiple genres for each review
    df_genres = pd.get_dummies(df_genres, columns=['genre']).groupby('reviewid').sum().reset_index()

    #merge tables
    df = df.merge(df_years)
    df = df.merge(df_genres)
    df = df.merge(df_reviews).reset_index()
    print 'data merged'
    

In [93]:
df = df.sample(50)

In [94]:
#Fill NA Values for Year, Drop any duplicates, drop any unimportant columns
df.year.fillna(value=df.pub_year, inplace=True)
df.drop_duplicates('reviewid',inplace=True)
df.drop(['index'], axis=1, inplace=True)
df.drop('author_type', axis=1, inplace=True)
df.year = df.year.astype(int)
df = df[df['pub_year'] < 2018]

list_of_words = ['reissue', 'remaster', 'box set', 'collector', 'delux']
parse_content_for_reissue(df, list_of_words)
df = reissue_columns(df)
df.reset_index(inplace=True)
df['content'] = df['content'].map(remove_weird_char)
df['abstract'] = df['abstract'].map(remove_weird_char)
print df.shape
print 'beginning NLP parsing'
df = parse_for_adj(df, 'abstract', 'abstract_adj')
print 'abs adj done'
print df.shape
df = parse_for_adj(df, 'content', 'content_adj')
print 'content adj done'
print df.shape
df = add_sentiment(df, 'content', 'cont_polarity', 'cont_subjectivity')
print 'sentiment content done'
print df.shape
df = add_sentiment(df, 'abstract', 'abs_polarity', 'abs_subjectivity')
print 'abstract sentiment done'
print df.shape
df = add_sentiment(df, 'abstract_adj', 'abs_adj_polarity', 'abs_adj_subjectivity')
print 'abstract_adj done'
print df.shape

In [95]:
df.head()

,index,reviewid,title,artist,url,score,best_new_music,best_new_reissue,author,pub_date,...,num_years_since_release,new_album,abstract_adj,content_adj,cont_polarity,cont_subjectivity,abs_polarity,abs_subjectivity,abs_adj_polarity,abs_adj_subjectivity
0,17948,5929d89dd7a71d1ea56999f4,shine,daniel lanois,https://pitchfork.com/reviews/albums/4950-shine/,5.6,0,0,brandon stosuy,2003-05-18,...,0,1,third first old difficult,third first old difficult third first old diff...,0.066053,0.507326,-0.037500,0.383333,-0.0375,0.383333
1,15171,5929dfa1c0084474cd0c4a70,"shinola, vol. 1",ween,https://pitchfork.com/reviews/albums/8610-shin...,7.6,0,0,adam moerder,2005-09-21,...,0,1,,experimental stale ersatz first sophomoric lit...,0.150579,0.510283,-0.166667,0.166667,0.0000,0.000000
2,18577,5929da35d7a71d1ea5699b91,the golden vessyl of sound,yume bitsu,https://pitchfork.com/reviews/albums/8836-the-...,8.5,0,0,mark richardson,2002-10-03,...,0,1,,last powerful spacy several ethereal catchy en...,0.238690,0.534364,0.000000,0.000000,0.0000,0.000000
3,6170,5929e73ad7a71d1ea569a7ce,news from nowhere,darkstar,https://pitchfork.com/reviews/albums/17575-new...,7.0,0,0,mark pytlik,2013-02-08,...,0,1,second full electronic dreampop charming,second full electronic dreampop charming dream...,0.199413,0.538180,0.350000,0.516667,0.3500,0.516667
4,19611,5929dc9f13d197565213cfcd,re:volution ep,coldcut,https://pitchfork.com/reviews/albums/1533-revo...,3.7,0,0,james p. wisdom,2001-09-18,...,0,1,short sweet,short sweet short sweet countless delicious cr...,0.132697,0.469314,0.337500,0.487500,0.1750,0.475000


In [84]:
df['content_adj']

0     musician direct early direct electronic techno...
1     excellent balcony molten worthy frantic smooth...
2     restive much easy natural tough creative tradi...
3     heated cool effective musical worn modern comm...
4     Former former fair psych green cohesive hard p...
5     democratic sampled sliced diced sampled origin...
6     compelling unifying clear former debut definit...
7     suitable actual actual suitable typical friend...
8     old Ukrainian igneous cold remote formative su...
9     familiar sick good new major forward electroni...
10    Second bright acoustic second surprising slow ...
11    Italian new classical romantic Precious first ...
12    certifiable wondrous human dreamy much recreat...
13    Subtitled wonderful Caribbean West African fre...
14    strong second Australian epic dark literate ri...
15    new electronic odd last harmolodic former Japa...
16    electronic middle mature mannered late wild yo...
17    long English late slow spare huge easy pow

In [89]:
df1 = df

In [96]:
df = add_sentiment(df, 'content_adj', 'cont_adj_polarity', 'cont_adj_subjectivity')
print 'content_adj done'
print df.shape

print 'sentiment abs done'
df = add_columns(df)
print df.shape

In [97]:
df.head()

,index,reviewid,title,artist,url,score,best_new_music,best_new_reissue,author,pub_date,...,abs_polarity,abs_subjectivity,abs_adj_polarity,abs_adj_subjectivity,cont_adj_polarity,cont_adj_subjectivity,word_count,adj_count,adj_freq,score_bin
0,17948,5929d89dd7a71d1ea56999f4,shine,daniel lanois,https://pitchfork.com/reviews/albums/4950-shine/,5.6,0,0,brandon stosuy,2003-05-18,...,-0.037500,0.383333,-0.0375,0.383333,0.007482,0.520355,758,68,0.089710,5
1,15171,5929dfa1c0084474cd0c4a70,"shinola, vol. 1",ween,https://pitchfork.com/reviews/albums/8610-shin...,7.6,0,0,adam moerder,2005-09-21,...,-0.166667,0.166667,0.0000,0.000000,0.121599,0.569320,481,42,0.087318,7
2,18577,5929da35d7a71d1ea5699b91,the golden vessyl of sound,yume bitsu,https://pitchfork.com/reviews/albums/8836-the-...,8.5,0,0,mark richardson,2002-10-03,...,0.000000,0.000000,0.0000,0.000000,0.168398,0.557522,707,48,0.067893,8
3,6170,5929e73ad7a71d1ea569a7ce,news from nowhere,darkstar,https://pitchfork.com/reviews/albums/17575-new...,7.0,0,0,mark pytlik,2013-02-08,...,0.350000,0.516667,0.3500,0.516667,0.203827,0.577150,560,59,0.105357,7
4,19611,5929dc9f13d197565213cfcd,re:volution ep,coldcut,https://pitchfork.com/reviews/albums/1533-revo...,3.7,0,0,james p. wisdom,2001-09-18,...,0.337500,0.487500,0.1750,0.475000,0.093327,0.432112,434,48,0.110599,3


In [ ]:
def add_sentiment(df, column, new_col_pol, new_col_subj):
    sentiments = []
    for each in range(len(df['reviewid'])):
        sentiments.append((df['reviewid'].iloc[each], TextBlob(df[column][each]).sentiment[0], TextBlob(df[column][each]).sentiment[1]))
    #                           TextBlob(df_mid['content'][each], analyzer=NaiveBayesAnalyzer()).sentiment))

    df_cont_sent = pd.DataFrame(sentiments, columns=['reviewid', new_col_pol, new_col_subj])
    return df.merge(df_cont_sent)